In [1]:
import pandas as pd

In [6]:
train = pd.read_csv('ts_train.csv')
test = pd.read_csv('ts_test.csv')
items = pd.read_csv('ts_items.csv')

In [7]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [8]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [9]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [10]:
items.shape

(22170, 3)

In [11]:
train.shape

(2935849, 6)

In [12]:
test.shape

(214200, 3)

In [13]:
all_data = pd.merge(train, test, on='item_id', how='inner')

In [16]:
both = pd.merge(train, test, on=['item_id', 'shop_id'], how='inner')

In [17]:
both.shape

(1224439, 7)

In [18]:
both.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID
0,02.01.2013,0,59,22154,999.0,1.0,150983
1,03.01.2013,0,25,2574,399.0,2.0,56520
2,05.01.2013,0,25,2574,399.0,1.0,56520
3,07.01.2013,0,25,2574,399.0,1.0,56520
4,08.01.2013,0,25,2574,399.0,2.0,56520


In [19]:
shops = pd.read_csv('ts_shops.csv')

In [20]:
shops.shape

(60, 2)

In [21]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [26]:
shop_names_df = pd.merge(both, shops, on=['shop_id'], how='right')

In [27]:
shop_names_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,ID,shop_name
0,NaN,NaN,0,NaN,NaN,NaN,NaN,"!Якутск Орджоникидзе, 56 фран"
1,NaN,NaN,1,NaN,NaN,NaN,NaN,"!Якутск ТЦ ""Центральный"" фран"
2,16.01.2013,0.0,2,11391.0,899.0,1.0,22595.0,"Адыгея ТЦ ""Мега"""
3,16.02.2013,1.0,2,11391.0,899.0,1.0,22595.0,"Адыгея ТЦ ""Мега"""
4,13.02.2013,1.0,2,11391.0,899.0,1.0,22595.0,"Адыгея ТЦ ""Мега"""


In [28]:
shop_names_df.shape

(1224457, 8)

In [29]:
shop_names_df.isnull().sum()

date              18
date_block_num    18
shop_id            0
item_id           18
item_price        18
item_cnt_day      18
ID                18
shop_name          0
dtype: int64